In [6]:
import numpy as np
import data_util
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
DR_dataset_dir = 'diabetic_dataset'
X_train,Y_train = data_util.load_dr_dataset(DR_dataset_dir)
print('Training data shape',X_train.shape)
print('Training label shape',Y_train.shape)
batch_size,height,width,channels=X_train.shape
filters = np.random.rand(3,3,channels,3)*0.001
X_train = X_train - np.mean(X_train,axis = 0)
X_train = X_train.astype(np.float32)
print(type(X_train[0]))

Training data shape (10000, 32, 32, 3)
Training label shape (10000,)
<class 'numpy.ndarray'>


In [8]:
conv_filter_1 = tf.Variable(np.array(filters),dtype=np.float32,name='conv_filter_1')
conv_filter_2 = tf.Variable(np.array(filters),dtype=np.float32,name='conv_filter_2')
num_output_class = 10
X = tf.placeholder(tf.float32)
convolution_1 = tf.nn.conv2d(X,conv_filter_1,strides=[1,1,1,1],padding='SAME',name='convolution_1')
# with tf.Session() as sess:
#     output_1 = sess.run(convolution_1,{X:X_train[0:1,:,:,:]})
# print(output_1.shape)
# plt.imshow(output_1[0,:,:,0].astype('uint8'))
# plt.show()
output_1_pool = tf.nn.max_pool(convolution_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID",name='output_1_pool')
# with tf.Session() as sess:
#     output_1_pool = sess.run(max_pool,{X:output_1})
# print(output_1_pool.shape)
# plt.imshow(output_1_pool[0,:,:,0].astype('uint8'))
# plt.show()

convolution_2 = tf.nn.conv2d(output_1_pool,conv_filter_2,strides=[1,1,1,1],padding="SAME",name='convolution_2')
output_2 = tf.contrib.layers.flatten(convolution_2)
# with tf.Session() as sess:
#     sess.run(init)
#     output_2 = sess.run(convolution_2,{X:X_train[0:1,:,:,:]})

    
# output_2 = output_2.reshape(output_2.shape[0],-1)
# print(output_2.shape)

# with tf.Session() as sess:
#     output_2 = sess.run(convolution_2,{X:output_1_pool})
# print(output_2.shape)
# plt.imshow(output_2[0,:,:,0].astype('uint8'))
# plt.show()

# shapes = np.array(convolution_2).shape
# output_2 = np.array(convolution_2).reshape(shapes[0],-1)
weight = tf.Variable(np.random.randn(768,num_output_class),dtype=np.float32,name='full_conv_weight')

In [15]:
import time


tic = time.time()
final_output = tf.reshape(tf.nn.relu(tf.matmul(output_2,weight)),(10,),name='final_output')
# print(final_output)
# print(Y_train[0])

softmax = tf.nn.softmax(final_output)
softmax_loss = - tf.log(softmax)[Y_train[0]]
learning_rate = 5e-6
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(softmax_loss)
# gradient = tf.reshape(tf.gradients(softmax_loss,weight),(768,10))

# cross_entropy_gradient = (softmax_loss-1) * final_output

# weight_update = tf.assign(weight, weight - learning_rate * gradient)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
writer = tf.summary.FileWriter('./graphs/linear_reg', tf.get_default_graph())
with tf.Session() as sess:
#     saver.restore(sess,"/tmp/my_model_final.ckpt")
    sess.run(init)
    for i in range(100):
        sess.run(train,{X:X_train[0:1,:,:,:]})
        if i % 10 == 0:
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
            print(sess.run(softmax_loss,{X:X_train[0:1,:,:,:]}))
#     softmax_loss = - np.log(sess.run(softmax))[Y_train[0]]
toc = time.time()
print('time-delay:',toc-tic)

2.2255578
2.091898
1.8487083
1.443548
0.9361595
0.52025104
0.2899613
0.17861073
0.12154391
0.08915122
time-delay: 3.5497779846191406
